# Multi-Agent Travel Planning Orchestrator System
 
## Overview
This notebook demonstrates the **orchestrator pattern** where a central agent delegates tasks to specialized worker agents. Unlike sequential chaining, the orchestrator intelligently routes each request to the appropriate specialist.

<div align="center">
<img src="lesson_3.png" alt="Alt text" width="1050"/>
</div>
 
### Key Concepts Covered:
1. **Orchestrator Pattern**: Central agent routes requests to specialists
2. **Agent as Plugin**: Worker agents registered as plugins for the orchestrator
3. **Intelligent Delegation**: Orchestrator decides which agent handles each request
4. **ChatHistoryAgentThread**: Maintains conversation context across interactions
5. **Separation of Concerns**: Each agent has a narrow, focused responsibility

## 1. Setup and Configuration

In [1]:
import os
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Load environment variables
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_KEY")
url = os.getenv("URL")
api_version = "2024-12-01-preview"

## 2. Initialize Kernel
 
The kernel manages services and coordinates agent operations.

In [2]:
kernel = Kernel()

## 3. Configure Azure OpenAI Service
 
Create the chat completion service that will power all agents.

In [3]:
chat_service = AzureChatCompletion(
    deployment_name="none",  # Deployment name managed by base_url
    api_key=api_key,
    base_url=url,
    api_version=api_version  
)

# Register the service with the kernel
kernel.add_service(chat_service)

## 4. Create Specialized Worker Agents
 
These are **specialist agents** with narrow, focused expertise.
They will be used as **plugins** by the orchestrator agent.

### 4.1 Flight Booking Agent

In [4]:
flight_booking_agent = ChatCompletionAgent(
    service=chat_service,
    name="FlightBookingAgent",
    instructions="""
    AI Agent Persona: Flight Booking Specialist
    Role: A specialized assistant focused exclusively on flight bookings and airline information.
    Behavior: Only handles flight-related queries. Does not answer questions outside this domain.
    Response Style: Provides clear, actionable flight booking guidance.
    
    Agent Instructions:
    - When asked about booking flights, provide step-by-step guidance on checking availability, 
      comparing prices, and booking tickets.
    - Mention key considerations: dates, destinations, layovers, airline preferences, budget.
    - Always suggest checking multiple booking platforms (Google Flights, Kayak, airline websites).
    - Do NOT respond to questions about hotels, car rentals, attractions, or other travel topics.
    - If the query is outside flight booking, respond: "I only handle flight bookings. Please contact another specialist."
    """
)

### 4.2 Hotel Accommodation Agent

In [5]:
hotel_accommodation_agent = ChatCompletionAgent(
    service=chat_service,
    name="HotelAccommodationAgent",
    instructions="""
    AI Agent Persona: Hotel Accommodation Specialist
    Role: A specialized assistant focused exclusively on hotel bookings and lodging.
    Behavior: Only handles accommodation-related queries. Does not answer questions outside this domain.
    Response Style: Provides detailed hotel booking recommendations.
    
    Agent Instructions:
    - When asked about hotels or accommodations, provide guidance on finding and booking hotels.
    - Mention key factors: location, price range, amenities, reviews, cancellation policies.
    - Suggest platforms like Booking.com, Hotels.com, Airbnb for different travel styles.
    - Recommend checking reviews on TripAdvisor before booking.
    - Do NOT respond to questions about flights, attractions, car rentals, or other travel topics.
    - If the query is outside hotel booking, respond: "I only handle hotel accommodations. Please contact another specialist."
    """
)

### 4.3 Local Attractions Agent

In [6]:
local_attractions_agent = ChatCompletionAgent(
    service=chat_service,
    name="LocalAttractionsAgent",
    instructions="""
    AI Agent Persona: Local Attractions & Activities Specialist
    Role: A specialized assistant focused exclusively on tourist attractions and activities.
    Behavior: Only handles queries about things to do, attractions, and local experiences. 
    Response Style: Provides engaging recommendations for sightseeing and activities.
    
    Agent Instructions:
    - When asked about attractions or things to do, provide curated recommendations.
    - Categorize suggestions: historical sites, museums, outdoor activities, food experiences, nightlife.
    - Mention practical details: opening hours, ticket prices, best times to visit, booking requirements.
    - Suggest researching on TripAdvisor, GetYourGuide, or local tourism websites.
    - Do NOT respond to questions about flights, hotels, car rentals, or other travel logistics.
    - If the query is outside attractions/activities, respond: "I only handle local attractions and activities. Please contact another specialist."
    """
)

## 5. Create Orchestrator Agent
 
The **orchestrator** is the key innovation here. It:
- Does NOT answer questions directly
- Analyzes each request to determine the appropriate specialist
- Delegates to worker agents (registered as plugins)
- Routes requests intelligently based on query content

In [7]:
orchestrator_agent = ChatCompletionAgent(
    service=chat_service,
    name="TravelOrchestratorAgent",
    instructions="""
    AI Agent Persona: Travel Planning Coordinator
    Role: A management assistant that intelligently routes travel requests to specialized agents.
    Behavior: NEVER answers user questions directly. Always delegates to the appropriate specialist agent.
    Response Style: Routes requests by invoking the correct plugin/agent.
    
    Agent Instructions:
    DO NOT answer any questions directly yourself.
    
    Delegation Rules:
    If the request is about FLIGHTS, AIRLINES, or FLIGHT BOOKING → invoke FlightBookingAgent
    If the request is about HOTELS, ACCOMMODATIONS, or LODGING → invoke HotelAccommodationAgent
    If the request is about ATTRACTIONS, ACTIVITIES, or THINGS TO DO → invoke LocalAttractionsAgent
    
    If the request is about topics NONE of your specialists handle (e.g., travel insurance, car rentals, 
       visa requirements), respond: "I apologize, but I don't have a specialist for that type of request. 
       My team covers flights, hotels, and local attractions only."
    
    CRITICAL: You are a router, not an answerer. Always delegate to plugins.
    """,
    plugins=[flight_booking_agent, hotel_accommodation_agent, local_attractions_agent],
)

## 6. Create Chat Thread
 
The **ChatHistoryAgentThread** maintains conversation history.
This allows agents to reference previous messages in the conversation.

In [8]:
thread = ChatHistoryAgentThread()

## 7. Test the Orchestrator with Different Queries
 
We'll test various travel-related queries to see how the orchestrator routes them.

In [11]:
async def main():
    """
    Tests the orchestrator agent with different types of travel queries.
    
    Expected Behavior:
    - Flight queries → FlightBookingAgent handles
    - Hotel queries → HotelAccommodationAgent handles
    - Attraction queries → LocalAttractionsAgent handles
    - Out-of-scope queries → Orchestrator declines politely
    """
    
    # Test queries covering different specialist domains
    prompts = [
        "How do I book a flight to Paris?",
        "What are the best hotels in Tokyo?",
        "What attractions should I visit in Rome?",
        "How do I rent a car for my trip?",
        "I need travel insurance recommendations"
    ]

    print("=" * 70)
    print("TRAVEL PLANNING ORCHESTRATOR SYSTEM")
    print("=" * 70)
    print("\nTesting orchestrator delegation with various travel queries...\n")

    for index, prompt in enumerate(prompts, start=1):
        print("-" * 70)
        print(f"Query #{index}: {prompt}")
        print("-" * 70)
        
        # Invoke orchestrator - it will delegate to appropriate specialist
        async for message in orchestrator_agent.invoke(prompt):
            print(f"\n📋 Response:\n{message}\n")

## 8. Run the Demo

In [12]:
await main()

TRAVEL PLANNING ORCHESTRATOR SYSTEM

Testing orchestrator delegation with various travel queries...

----------------------------------------------------------------------
Query #1: How do I book a flight to Paris?
----------------------------------------------------------------------

📋 Response:
I've routed your request to the Flight Booking Specialist for guidance on booking a flight to Paris. If you need help with hotels or things to do in Paris, let me know and I'll connect you with the right specialist.

----------------------------------------------------------------------
Query #2: What are the best hotels in Tokyo?
----------------------------------------------------------------------

📋 Response:
Your request is about hotels in Tokyo. I am routing this to the Hotel Accommodation Specialist for the best recommendations.

----------------------------------------------------------------------
Query #3: What attractions should I visit in Rome?
------------------------------------

## Key Takeaways
 
### 1. **Orchestrator Pattern vs Sequential Chaining**
 
| **Sequential Chain** | **Orchestrator Pattern** |
|---------------------|-------------------------|
| All agents process every request | Only relevant agent processes each request |
| Output flows through all agents | Direct routing to specialist |
| A → B → C → D | Orchestrator → {A OR B OR C} |
| Good for: Progressive refinement | Good for: Discrete task types |
 
### 2. **Agents as Plugins**
This is a powerful pattern:
```python
orchestrator_agent = ChatCompletionAgent(
     ...,
     plugins=[specialist1, specialist2, specialist3]
)
```
The orchestrator can **invoke** these agents as if they were functions!
 
### 3. **Separation of Concerns**
Each agent has a single responsibility:
- **FlightBookingAgent**: Only flights
- **HotelAccommodationAgent**: Only hotels
- **LocalAttractionsAgent**: Only attractions
- **Orchestrator**: Only routing (never answers directly)
 
This makes the system:
-  Easier to test (test each specialist independently)
-  Easier to maintain (update one specialist without affecting others)
-  Easier to extend (add new specialists without changing orchestrator logic)
 
### 4. **ChatHistoryAgentThread**
Maintains conversation context:
```python
thread = ChatHistoryAgentThread()
```
This allows for multi-turn conversations where agents can reference previous exchanges.
 
### 5. **When to Use This Pattern**
 
 **Use Orchestrator Pattern when:**
 - Requests fall into discrete categories
 - Different requests need different specialists
 - You want to minimize unnecessary processing
 - Specialists have non-overlapping expertise
 
 **Don't use when:**
 - All requests need the same processing pipeline
 - Agents need to build on each other's work
 - Request categorization is ambiguous
 
### 6. **Production Enhancements**
 
**Smart Routing:**
```python
 # Use an LLM to classify the query first
 classification_agent = ChatCompletionAgent(...)
 category = classification_agent.classify(query)
 appropriate_agent = agent_map[category]
```
 
**Fallback Handling:**
```python
Add a general-purpose agent for edge cases
general_agent = ChatCompletionAgent(
     name="GeneralAgent",
     instructions="Handle miscellaneous travel questions..."
)
```
 
 **Multi-Agent Responses:**
 ```python
 # Some queries might need multiple specialists
 if query.needs_multiple_agents():
     results = await asyncio.gather(
         flight_agent.invoke(query),
         hotel_agent.invoke(query)
     )
 ```
 
 **Confidence Scoring:**
 ```python
 # Have orchestrator get confidence from each agent
 # Route to the agent with highest confidence
 confidences = [agent.assess_relevance(query) for agent in agents]
 best_agent = agents[argmax(confidences)]
 ```
 
 ### 7. **Real-World Use Cases**
 
 This pattern works well for:
 - **Customer Service**: Route to billing, tech support, returns, etc.
 - **Content Creation**: Route to writer, editor, designer, etc.
 - **Data Analysis**: Route to cleaning, visualization, statistics, etc.
 - **IT Helpdesk**: Route to network, hardware, software specialists
 - **Medical Triage**: Route to appropriate medical specialist
 
 ### 8. **Comparison: All Three Patterns**
 
 **Pattern 1: Independent Agents** (First notebook)
 - All agents answer every query
 - Good for: Showing different perspectives
 
 **Pattern 2: Sequential Chain** (Second notebook)  
 - Agents process in order, building on each other
 - Good for: Progressive refinement
 
 **Pattern 3: Orchestrator** (This notebook)
 - Central router delegates to specialists
 - Good for: Discrete task categories
